In [38]:

from openai import OpenAI
import mysql.connector
from pydantic import BaseModel, Field
from typing import List
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

client = OpenAI(api_key='')

In [2]:
exp_base= client.files.create(
    file= open('experiencia.pdf', 'rb'),
    purpose= 'user_data'
)
exp_lic1= client.files.create(
    file= open('exp1.pdf', 'rb'),
    purpose= 'user_data'
)
exp_lic2= client.files.create(
    file= open('exp2.pdf', 'rb'),
    purpose= 'user_data'
)
exp_lic3= client.files.create(
    file= open('exp3.pdf', 'rb'),
    purpose= 'user_data'
)

In [6]:
# Conexión a la base de datos:
class ConexionMySQL:
    def __init__(self, host, port, user, password, database):
        self.host = host
        self.port = port
        self.user = user
        self.password = password
        self.database = database

        # Aquí iría la lógica para conectarse a MySQL
        self.conexion = mysql.connector.connect(
            host=self.host,
            port=self.port,
            user=self.user,
            password=self.password,
            database=self.database
        )
        self.cursor = self.conexion.cursor()
        if self.conexion.is_connected():
            # Verificar la conexión
            print("Conexión exitosa a la base de datos MySQL")
        else:
            print("Error al conectar a la base de datos MySQL")

In [ ]:
# Pruebas con DB
listas = []
def main_mysql():
    # Crear una instancia de la clase ConexionMySQL
    db_mysql = ConexionMySQL(
        host="localhost",
        port=3306,
        user = "root",
        password = "admin",
        database = "db_insitel"
    )

    # Ejecutar una consulta a la base de datos
    db_mysql.cursor.execute("SELECT objeto FROM experiencia;")

    # Obtener los resultados
    resultados = db_mysql.cursor.fetchall()
    # Imprimir los resultados
    lista = [fila[0] for fila in resultados]
    # Cerrar la conexión
    db_mysql.cursor.close()
    db_mysql.conexion.close()
    print("Conexión cerrada")
    return lista
main_mysql()

In [22]:
array_empty = []
listas = main_mysql()
print(listas)
print('**********')
for list in listas:

    response2 = client.embeddings.create(
      model="text-embedding-3-small",
      input=list
    )
    embeding =  response2.data[0].embedding
    array_empty.append(embeding)
print(array_empty)

Conexión exitosa a la base de datos MySQL
Conexión cerrada
['Suministro DDP e instalación de equipos de entretenimiento (audiovisuales), redes eléctricas, de datos y alarmas, descritos en el listado de precios unitarios de este documento, para la respuesta en funcionamiento de los Puntos Vive Digital ubicados en los municipios de Monteria, Sincelejo, Valledupar, Barrancabermeja, Apartado, Turbo, Puerto Berrio y Santa Rosa de Osos.', 'Suministro, instalación, configuración y puesta en marcha de la Red WiFi de las Oficinas de INPRO DE COLOMBIA S.A.S.  en la Ciudad de Bogotá.', 'Suministro, instalación, configuración y puesta en servicio de equipos activos de red para la infraestructura de ERT de Cali, Tulua y Palmira.', 'Prestación del Servicio profesional para el apoyo y soporte en la elaboración, por parte de la entidad, del diseño del sistema de seguridad integral e inteligente de CAMPROVIMPO SIO', 'El contratista se obliga para con EMTEL S.A. E.S.P. a realizar “Suministro e instalaci

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [35]:
lic1 = 'Adquirir la infraestructura de switches de red, incluidos los servicios de instalación, configuración, migración y puesta en producción, para actualizar los componentes de red local y de área amplia, mejorando los servicios básicos de tecnologías de información del IDEAM, en el marco del convenio No. 9677-PPAL001-369-2023 celebrado entre el FNGRD y el IDEAM.'
lic2 = 'PRESTACION DE SERVICIOS PROFESIONALES DE ENFERMERIA PARA LA CONFORMACION DEL EQUIPO BASICO EN SALUD N° 11 SEGÚN LA RESÓLUCION 1499 DÉ 26 DE AGOSTO DE 2024, EN LA ZONA URBANA DEL MUNICIPIO VALLE DEL GUAMUEZ DEL DEPARTAMENTO DEL PUTUMAYO.'
response3 = client.embeddings.create(
  model="text-embedding-3-small",
  input=lic2
)
embeding2 =  response3.data[0].embedding

query_embedding = np.array(embeding2)

In [36]:
sims = cosine_similarity([query_embedding], array_empty)[0]
top_indices = sims.argsort()[-3:][::-1]


In [37]:
text_similares = []
for idx in top_indices:
    print(f"Indice: {idx}, Similaridad: {sims[idx]:.4f}, Texto: {listas[idx]}")
    text_similares.append(listas[idx])

print(text_similares)

Indice: 3, Similaridad: 0.4849, Texto: Prestación del Servicio profesional para el apoyo y soporte en la elaboración, por parte de la entidad, del diseño del sistema de seguridad integral e inteligente de CAMPROVIMPO SIO
Indice: 132, Similaridad: 0.4779, Texto: EL CONTRATISTA SE COMPROMETE CON UNIMOS S.A. E.S.P. A REALIZAR EL SUMINISTRO DE EQUIPOS, ELEMENTOS DE REDES DE DATOS Y REDES ELÉCTRICAS, NECESARIOS PARA DAR CUMPLIMIENTO A CONVENIOS Y CONTRATOS INTERADMINISTRATIVOS CELEBRADOS CON LA
ALCALDÍA MUNICIPAL DE IPIALES.
Indice: 46, Similaridad: 0.4725, Texto: Suministro de bienes y servicios requeridos para el desarrollo de soluciones e infraestructura de telecomunicaciones, con la finalidad de ampliar la cobertura de infraestructura de telecomunicaciones así como el establecimiento de centros de acceso comunitario a internet, para el DEPARTAMENTO DE CUNDINAMARCA, en 300 instituciones publicas y educativas
['Prestación del Servicio profesional para el apoyo y soporte en la elaboración,

In [3]:
class ComparacionDescripcion(BaseModel):
    palabras_similares: List[str] = Field(description="Palabras o frases similares encontradas en ambos archivos")
    porcentaje_similitud: float = Field(description="Porcentaje de similitud entre la descripción solicitada y las de la base")

In [4]:
response = client.responses.create(
    model='gpt-4o-mini',
    input=[
        {
            'role': 'user',
            'content': [
                {'type': 'input_file', 'file_id': exp_base.id},
                {'type': 'input_file', 'file_id': exp_lic3.id},
                {'type': 'input_text', 'text': (
                    f'Compara el objeto solicitado {text_similares}en el archivo de licitación con las descripciones de objetos disponibles en la base de datos, tener en cuenta que somos INSITEL, una empresa de telecomunicaciones, ciberseguridad y automatización.'
                    'Para cada objeto solicitado, identifica las palabras o frases más similares entre ambos docuemntos que se relacionen entre si.'
                    'Calcula un porcentaje de similitud basado en coincidencia semántica, de contexto y área de trabajo. '
                    'Devuélveme solo una lista JSON, con esta estructura exacta por cada objeto solicitado:\n\n'
                    '[\n'
                    '  {\n'
                    '    "palabras_similares": ["servicio técnico", "mantenimiento de hardware"],\n'
                    '    "porcentaje_similitud": 87.5\n'
                    '  },\n'
                    '  {\n'
                    '    "palabras_similares": ["salud", "cocina", "deportes"],\n'
                    '    "porcentaje_similitud": 10.3\n'
                    '  },\n'
                    ']\n\n'
                    "No incluyas explicaciones ni texto adicional, solo el arreglo JSON."
                )}
            ]
        }
    ]
)

In [5]:
salida = response.output_text.strip()

try:
    lista_dicts = json.loads(salida)
    comparaciones = [ComparacionDescripcion(**item) for item in lista_dicts]
    for comp in comparaciones:
        print(comp)
except Exception as e:
    print("Error al convertir la salida en objetos ComparacionDescripcion: ", e)
    print("Respuesta del modelo:\n", salida)


Error al convertir la salida en objetos ComparacionDescripcion:  Expecting value: line 1 column 1 (char 0)
Respuesta del modelo:
 ```json
[
  {
    "palabras_similares": ["servicios profesionales", "equipo básico en salud"],
    "porcentaje_similitud": 75.0
  },
  {
    "palabras_similares": ["enfermería", "atención en salud"],
    "porcentaje_similitud": 25.0
  }
]
```
